In [16]:
import requests
import pandas as pd
import numpy as np
import json

In [17]:
url = "https://deribit.com/api/v2/public/get_book_summary_by_currency?currency=BTC&kind=option"

r = requests.get(url)

df = pd.DataFrame(json.loads(r.text))

In [18]:
dff = pd.DataFrame.from_dict(df['result'][0], orient='index').T
for i in range(1, len(df)):
    dff = dff.append(df['result'][i], ignore_index=True)

In [19]:
dff.tail()

,volume,underlying_price,underlying_index,quote_currency,price_change,open_interest,mid_price,mark_price,low,last,interest_rate,instrument_name,high,estimated_delivery_price,creation_timestamp,bid_price,base_currency,ask_price
519,15.9,16483.58,BTC-30JUN23,BTC,-1.626,310.2,0.06125,0.060793,0.0605,0.0605,0.0,BTC-30JUN23-12000-P,0.0615,16568.8,1672492422854,0.0595,BTC,0.063
520,46.0,16502.08,BTC-31MAR23,BTC,-5.2632,2514.3,0.02825,0.028265,0.027,0.027,0.0,BTC-31MAR23-21000-C,0.031,16568.8,1672492422854,0.0275,BTC,0.029
521,4.2,16536.04,BTC-24FEB23,BTC,-13.3333,166.4,0.00625,0.006078,0.0065,0.0065,0.0,BTC-24FEB23-10000-P,0.0075,16568.8,1672492422854,0.006,BTC,0.0065
522,0.0,16514.08,BTC-29SEP23,BTC,None,69.6,0.30375,0.307323,None,0.3,0.0,BTC-29SEP23-19000-P,None,16568.8,1672492422854,0.25,BTC,0.3575
523,0.0,16567.54,BTC-6JAN23,BTC,None,132.7,None,0.000133,None,0.0005,0.0,BTC-6JAN23-20000-C,None,16568.8,1672492422854,None,BTC,0.0005


In [13]:
from datetime import datetime
t = []
for i in range(0, len(dff)):
    ts = int(dff['creation_timestamp'][i])
    t.append(datetime.utcfromtimestamp(ts / 1000).strftime('%Y-%m-%d %H:%M:%S')) 
    
dff['t'] = t
first_column = dff.pop('t')
dff.insert(0, 't', first_column)

In [14]:
dff.head()

,t,volume,underlying_price,underlying_index,quote_currency,price_change,open_interest,mid_price,mark_price,low,last,interest_rate,instrument_name,high,estimated_delivery_price,creation_timestamp,bid_price,base_currency,ask_price
0,2022-12-23 07:02:32,10.9,16814.73,BTC-30DEC22,BTC,0.0,2007.6,None,0.000193,0.0005,0.0005,0.0,BTC-30DEC22-11000-P,0.0005,16837.39,1671778952702,NaN,BTC,0.0005
1,2022-12-23 07:02:32,0.0,16720.84,BTC-30JUN23,BTC,None,4.4,0.603,0.601476,None,0.625,0.0,BTC-30JUN23-26000-P,None,16837.39,1671778952702,0.512,BTC,0.694
2,2022-12-23 07:02:32,0.9,16767.350003,SYN.BTC-13JAN23,BTC,0.0,0.9,0.00225,0.002239,0.002,0.002,0.0,BTC-13JAN23-21000-C,0.002,16837.39,1671778952702,0.002,BTC,0.0025
3,2022-12-23 07:02:32,0.0,16717.93,BTC-31MAR23,BTC,None,1294.4,0.007,0.006984,None,0.01,0.0,BTC-31MAR23-35000-C,None,16837.39,1671778952702,0.0065,BTC,0.0075
4,2022-12-23 07:02:32,0.0,16717.93,BTC-31MAR23,BTC,None,0.0,None,13.953955,None,None,0.0,BTC-31MAR23-250000-P,None,16837.39,1671778952702,None,BTC,None


In [7]:
!pip install tardis-client
import asyncio
from tardis_client import TardisClient, Channel
tardis_client = TardisClient(api_key="YOUR_API_KEY")

async def replay():
  # replay method returns Async Generator
  messages = tardis_client.replay(
    exchange="deribit",
    from_date="2019-07-01",
    to_date="2019-07-02",
    filters=[Channel(name="book", symbols=["BTC-PERPETUAL"])]
  )

  # messages as provided by Deribit real-time stream
  async for local_timestamp, message in messages:
    print(message)


asyncio.run(replay())

RuntimeError: asyncio.run() cannot be called from a running event loop

In [20]:
import asyncio
import json
import pandas as pd
import websockets
from dateutil import parser
import datetime as dt


class DeribitOptionsData:
    def __init__(self, instrument):
        instrument = instrument.upper()
        if instrument not in ['BTC', 'ETH']:
            raise ValueError('instrument must be either BTC or ETH')
        self._instrument = instrument
        self.options = None
        self._get_options_chain()
        self.call_time = dt.datetime.now()

    @staticmethod
    async def call_api(msg):
        async with websockets.connect('wss://www.deribit.com/ws/api/v2') as websocket:
            await websocket.send(msg)
            while websocket.open:
                response = await websocket.recv()
                return response

    @staticmethod
    def json_to_dataframe(response):
        response = json.loads(response)
        results = response['result']
        df = pd.DataFrame(results)
        return df

    def update(self):
        self._get_options_chain()

    @property
    def instrument(self):
        return self._instrument

    @instrument.setter
    def instrument(self, new_instrument):
        if isinstance(new_instrument, str):
            self._instrument = new_instrument
        else:
            raise ValueError('New instrument must be a string')

    @staticmethod
    def date_parser(str_date):
        date = str_date.split('-')[-1]
        return parser.parse(date)

    @staticmethod
    def strike_parser(inst_name):
        strike = inst_name.split('-')[-2]
        return int(strike)

    @staticmethod
    def side_parser(inst_name):
        side = inst_name.split('-')[-1]
        if side == 'P':
            return 'Put'
        if side == 'C':
            return 'Call'
        else:
            return 'N/A'

    @staticmethod
    def async_loop(message):
        return asyncio.get_event_loop().run_until_complete(DeribitOptionsData.call_api(message))


    def process_df(self, df):
        # add expiry column
        df['expiry'] = [DeribitOptionsData.date_parser(date) for date in df.underlying_index]
        #add strike column
        df['strike'] = [DeribitOptionsData.strike_parser(i) for i in df.instrument_name]

        # add side, i.e. put or call
        df['type'] = [DeribitOptionsData.side_parser(j) for j in df.instrument_name]

        # get example option with closest expiry in order to calc dollar price of options_trading

        # spot = DeribitOptionsData.get_quote(self._instrument)

        df['dollar_bid'] = df.underlying_price * df.bid_price
        df['dollar_ask'] = df.underlying_price * df.ask_price
        df['dollar_mid'] = df.underlying_price * df.mid_price

        #create time to expiry as float
        df['time'] = (df['expiry'] - dt.datetime.now()).dt.days / 365

        return df

    @staticmethod
    def get_quote(instrument):
        msg1 = \
            {
                "jsonrpc": "2.0",
                "id": 9344,
                "method": "public/ticker",
                "params": {
                    "instrument_name": instrument +'-PERPETUAL',
                    "kind": "future"
                }
            }
        quote = json.loads(DeribitOptionsData.async_loop(json.dumps(msg1)))
        return float(quote['result']['last_price'])

    @property
    def chain(self):
        return self.options

    def _get_options_chain(self):
        msg1 = \
            {
                "jsonrpc": "2.0",
                "id": 9344,
                "method": "public/get_book_summary_by_currency",
                "params": {
                    "currency": self._instrument,
                    "kind": "option"
                }
            }

        response = self.async_loop(json.dumps(msg1))
        df = self.json_to_dataframe(response)
        df = self.process_df(df)
        self.options = df.copy(deep=True)

    def available_instruments(self, currency, expired=False):
        msg = \
            {
                "jsonrpc": "2.0",
                "id": 9344,
                "method": "public/get_instruments",
                "params": {
                    "currency": currency,
                    "kind": "option",
                    "expired": expired
                }
            }
        resp = self.async_loop(json.dumps(msg))
        resp = json.loads(resp)
        instruments = [d["instrument_name"] for d in resp['result']]
        return instruments

    @classmethod
    def option_info(cls, option_label):
        msg = \
            {
                "jsonrpc": "2.0",
                "id": 8106,
                "method": "public/ticker",
                "params": {
                    "instrument_name": option_label
                }
            }

        response = DeribitOptionsData.async_loop(json.dumps(msg))
        return json.loads(response)

    def expiries(self):
        return sorted(self.options.expiry.unique())

    def get_side_exp(self, side, exp='all'):
        if side.capitalize() not in ['Call', 'Put']:
            raise ValueError("Side must be 'Call' or 'Put'")
        if exp == 'all':
            return self.options[self.options['type'] == side]
        else:
            return self.options[(self.options.expiry == exp) & (self.options['type'] == side)]


In [21]:
optObj = DeribitOptionsData('ETH')
opts = optObj.options

print(opts.columns)

RuntimeError: This event loop is already running